## Load packages

In [1]:
import numpy as np
import pandas as pd
import math
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding1D, BatchNormalization, Flatten, Conv1D
from keras.layers import AveragePooling1D, MaxPooling1D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
# from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

Using TensorFlow backend.


## Load dataset

In [2]:
# import Matt's 'chr1_ukb_X.npz' file
chr1_ukb_X = np.load('/home/magu/deepmix/data/ALL_DNA_dataset/chr1_1kg_X.npz')
# create dict mapping from sampleid to superpopulation
sampleid_to_superpopulation_pandas_df = pd.read_csv('/home/jsokol/Data/igsr_samples.tsv', delimiter="\t")
sampleid_to_superpopulation_dict = sampleid_to_superpopulation_pandas_df.set_index('Sample name')['Superpopulation code'].to_dict()
# remove samples from chr1_ukb_X that are not in dict or that do no contain value
temp = set()
for x in np.nditer(chr1_ukb_X['S']):
    sample_id = str((x.item(0))).replace("'", "").replace("b", "").split("_")[0]
    if sample_id not in sampleid_to_superpopulation_dict:
        print(sample_id + ' is not in dict. Make sure to remove this sample from the dataset.')
    elif sampleid_to_superpopulation_dict[sample_id] == float('nan'):
        print(sample_id + ' maps to nan in dict')
chr1_ukb_X_G = chr1_ukb_X['G'][:,:,:]
chr1_ukb_X_V = chr1_ukb_X['V'][:,:]

In [3]:
# define number of classes
classes = [0, 1, 2, 3, 4]
# flatten input data
X_all_data = chr1_ukb_X_G.reshape(chr1_ukb_X_G.shape[0], -1).T

In [4]:
# one hot encode labels
Y_all_data = np.zeros((5, 5008))
for i in range(5008):
    sample_id = str(chr1_ukb_X['S'][i].item(0)).replace("'", "").replace("b", "").split("_")[0]
    if sampleid_to_superpopulation_dict[sample_id] == 'AFR':
        Y_all_data[0,i] = 1
    elif sampleid_to_superpopulation_dict[sample_id] == 'AMR':
        Y_all_data[1,i] = 1
    elif sampleid_to_superpopulation_dict[sample_id] == 'EAS':
        Y_all_data[2,i] = 1
    elif sampleid_to_superpopulation_dict[sample_id] == 'EUR':
        Y_all_data[3,i] = 1
    elif sampleid_to_superpopulation_dict[sample_id] == 'SAS':
        Y_all_data[4,i] = 1

In [5]:
# randomly shuffle the order of the data
randomize = np.arange(X_all_data.shape[1])
np.random.shuffle(randomize)
X_all_data = X_all_data[:,randomize]
Y_all_data = Y_all_data[:,randomize]

In [69]:
# split data into training and test sets
X_train = X_all_data[:,:4000]
X_test = X_all_data[:,4000:]
Y_train = Y_all_data[:,:4000]
Y_test = Y_all_data[:,4000:]
# convert to correct format for use in keras
X_train = X_train[:,:,np.newaxis]
X_train = np.transpose(X_train, (1, 0, 2))
X_test = X_test[:,:,np.newaxis]
X_test = np.transpose(X_test, (1, 0, 2))
Y_train = np.transpose(Y_train, (1, 0))
Y_test = np.transpose(Y_test, (1, 0))

## Build model

In [71]:
def model2(n_snps_per_sample):
    """
    input_shape: The height, width and channels as a tuple.  
        Note that this does not include the 'batch' as a dimension.
        If you have a batch like 'X_train', 
        then you can provide the input_shape using
        X_train.shape[1:]
    """
    
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(shape=(n_snps_per_sample, 1))

    # First convolutional block
    X = Conv1D(filters=49, kernel_size=500 ,strides=500, padding = 'valid', name = 'conv0')(X_input)
    X = BatchNormalization(axis = -1, name = 'bn0')(X)
    X = Activation('relu')(X)

    # Second convolutional block
    X = Conv1D(49, 75, strides = 5, padding = 'same', name = 'conv1')(X)
    X = BatchNormalization(axis = -1, name = 'bn1')(X)
    X = Activation('relu')(X)
    
    # MAXPOOL
    X = MaxPooling1D(2, name='max_pool')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(5, activation='softmax', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='model2')
    
    return model

## Create and compile the model 

In [72]:
# create model
model2 = model2(X_train.shape[1])
# compile model
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize model
model2.summary()

Model: "model2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 231504, 1)         0         
_________________________________________________________________
conv0 (Conv1D)               (None, 463, 49)           24549     
_________________________________________________________________
bn0 (BatchNormalization)     (None, 463, 49)           196       
_________________________________________________________________
activation_13 (Activation)   (None, 463, 49)           0         
_________________________________________________________________
conv1 (Conv1D)               (None, 93, 49)            180124    
_________________________________________________________________
bn1 (BatchNormalization)     (None, 93, 49)            196       
_________________________________________________________________
activation_14 (Activation)   (None, 93, 49)            0    

## Train model

In [74]:
model2.fit(X_train, Y_train, epochs = 10, batch_size = 128)

Epoch 1/10
4000/4000 [==============================] - 8s 2ms/step - loss: 0.2545 - accuracy: 0.9305
Epoch 2/10
4000/4000 [==============================] - 8s 2ms/step - loss: 0.0117 - accuracy: 0.9985
Epoch 3/10
4000/4000 [==============================] - 8s 2ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 4/10
4000/4000 [==============================] - 8s 2ms/step - loss: 7.8973e-04 - accuracy: 1.0000
Epoch 5/10
4000/4000 [==============================] - 8s 2ms/step - loss: 5.1609e-04 - accuracy: 1.0000
Epoch 6/10
4000/4000 [==============================] - 8s 2ms/step - loss: 3.8238e-04 - accuracy: 1.0000
Epoch 7/10
4000/4000 [==============================] - 8s 2ms/step - loss: 3.0450e-04 - accuracy: 1.0000: 0s - loss: 3.0646e-04 - accura
Epoch 8/10
4000/4000 [==============================] - 8s 2ms/step - loss: 2.4660e-04 - accuracy: 1.0000
Epoch 9/10
4000/4000 [==============================] - 8s 2ms/step - loss: 2.0349e-04 - accuracy: 1.0000
Epoch 10/10
4000/4000 [===

## Evaluate model

In [81]:
print('\n# Evaluate on test data')
results = model2.evaluate(X_test, Y_test, batch_size=1)
print('test loss, test acc:', results)


# Evaluate on test data
1008/1008 [==============================] - 3s 3ms/step
test loss, test acc: [0.05440417493529589, 0.9851190447807312]
